## Illustration of Xgboost method

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
import multiprocessing
from sklearn.model_selection import GridSearchCV
import os

In [11]:
main_dir = 'D:/Research/datasets/data_generated/LSTM_data/MS01/SR_data/EWRI_tutorial'

In [6]:
def XgboostParamTuning(X,y):
    if __name__ == "__main__":
        print("Parallel Parameter optimization")
        xgb_model = xgb.XGBRegressor(n_jobs=10, tree_method="hist",
                                    objective = 'reg:quantileerror', quantile_alpha=0.5, verbosity=0)
        clf = GridSearchCV(
            xgb_model,
            {"max_depth": [2, 4, 6, 8, 10, 15, 20], "n_estimators": [50, 100, 200], 'booster': ['gbtree', 'gblinear'],
             'eta': [0.10, 0.30, 0.50, 0.80, 1.00], 'subsample': [0.5, 0.75, 1.00]},
            verbose = 1,
            n_jobs = 2,
            cv = 3
        )
        clf.fit(X, y)
    
    return clf

In [12]:
# A plot to show how xgboost works
xtrain = np.arange(0, 100, 0.5).reshape(-1,1)
ytrain= (xtrain-50)**2 + 10

# Optimal hyperparameters
clf = XgboostParamTuning(xtrain, ytrain)
param = clf.best_params_

# Train the model
xgb_model = xgb.XGBRegressor(n_jobs=multiprocessing.cpu_count(), tree_method="hist", 
                             objective = 'reg:quantileerror', quantile_alpha=0.5, booster = param['booster'], max_depth = param['max_depth'], 
                             n_estimators = param['n_estimators'], eta = param['eta'], subsample = param['subsample'])
xgb_model.fit(xtrain, ytrain)

Parallel Parameter optimization
Fitting 3 folds for each of 630 candidates, totalling 1890 fits


XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.8, eval_metric=None,
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=50,
             n_jobs=48, num_parallel_tree=None, ...)

In [ ]:
os.makedirs(os.path.join(main_dir, 'xgb_fit_frames'), exist_ok=True)

# Get the predictions for different estimators


for i in range(1, xgb_model.n_estimators + 1):
    y_pred_stage = xgb_model.predict(xtrain, iteration_range=(0, i))

    plt.figure(figsize=(10, 6))
    plt.rcParams['font.family'] = 'Arial'
    plt.scatter(xtrain, ytrain, color='black')
    plt.scatter(xtrain, y_pred_stage, label=f"Boost {i}", s=6)
    plt.legend(['Observed', 'Predicted'], frameon = False)

    # Save frame
    frame_path = os.path.join(main_dir, f"xgb_fit_frames/frame_{i:03d}.png")
    plt.tight_layout()
    plt.savefig(frame_path)
    plt.close()